In [1]:
# skip-gram 은 직접 구현해볼까.... ㅎ 

# negative sampling 
# embedding layer 참조

# 참조링크
# https://reniew.github.io/22/
# https://towardsdatascience.com/implementing-word2vec-in-pytorch-skip-gram-model-e6bae040d2fb
# https://towardsdatascience.com/skip-gram-nlp-context-words-prediction-algorithm-5bbf34f84e0c


In [2]:
corpus = [
    'he is a king',
    'she is a queen',
    'he is a man',
    'she is a woman',
    'warsaw is poland capital',
    'berlin is germany capital',
    'paris is france capital',
]

def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

tokenized_corpus = tokenize_corpus(corpus)
print(tokenized_corpus)

[['he', 'is', 'a', 'king'], ['she', 'is', 'a', 'queen'], ['he', 'is', 'a', 'man'], ['she', 'is', 'a', 'woman'], ['warsaw', 'is', 'poland', 'capital'], ['berlin', 'is', 'germany', 'capital'], ['paris', 'is', 'france', 'capital']]


In [6]:
import pprint
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)
pprint.pprint(idx2word)
# print(idx2word)

{0: 'he',
 1: 'is',
 2: 'a',
 3: 'king',
 4: 'she',
 5: 'queen',
 6: 'man',
 7: 'woman',
 8: 'warsaw',
 9: 'poland',
 10: 'capital',
 11: 'berlin',
 12: 'germany',
 13: 'paris',
 14: 'france'}


In [11]:
import torch

window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = torch.Tensor(idx_pairs).long() # it will be useful to have this as numpy array

print(idx_pairs[:5])

tensor([[0, 1],
        [0, 2],
        [1, 0],
        [1, 2],
        [1, 3]])


In [31]:
import torch.nn.functional as F

def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

embedding_dims = 5
W1 = torch.randn(embedding_dims, vocabulary_size).float().requires_grad_()
W2 = torch.randn(vocabulary_size, embedding_dims).float().requires_grad_()

print(W1.shape)
print(type(W1))

print(vocabulary_size)

x = get_input_layer(3)
z1 = torch.matmul(W1, x)
z2 = torch.matmul(W2, z1)

log_softmax = F.log_softmax(z2, dim=0).view(1,-1)
y_true = torch.Tensor([3]).long()

print(log_softmax.shape)
print(y_true.shape)
loss = F.nll_loss(log_softmax.view(1,-1), y_true)

print(loss)

torch.Size([5, 15])
<class 'torch.Tensor'>
15
torch.Size([1, 15])
torch.Size([1])
tensor(9.6182, grad_fn=<NllLossBackward0>)


In [44]:
num_epochs = 100
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = get_input_layer(data).float().requires_grad_()
        y_true = torch.Tensor([target]).long()

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.item()
        
        loss.backward()
        W1.data = learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

Loss at epo 0: 2.7080401863370622
Loss at epo 10: 2.708040128435407
Loss at epo 20: 2.7080401522772655
Loss at epo 30: 2.708040087563651
Loss at epo 40: 2.7080400705337526
Loss at epo 50: 2.708040053503854
Loss at epo 60: 2.708040005820138
Loss at epo 70: 2.708039951324463
Loss at epo 80: 2.708039947918483
Loss at epo 90: 2.7080399206706454
